In [1]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI

import requests
import sys
import os
import logging
sys.path.append("..")
from utils import load_azd_env, set_azure_ai_project_connection_string, setup_logging, get_file_paths

In [2]:
load_azd_env()
set_azure_ai_project_connection_string()

In [3]:
enbale_logging = False

if enbale_logging:
    setup_logging()

In [4]:


with AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AZURE_AI_FOUNDRY_CONNECTION_STRING"],
) as project_client:

    # Explicit type hinting for IntelliSense
    client: AzureOpenAI = project_client.inference.get_azure_openai_client(
        # The latest API version is 2024-10-01-preview
        api_version = "2024-10-01-preview",
    )

    with client:
        agent = client.beta.assistants.create(
            model=os.environ["AZURE_AI_DEPLOYMENT_MODEL"], name="my-agent", instructions="You are a helpful agent"
        )
        print(f"Created agent, agent ID: {agent.id}")

        thread = client.beta.threads.create()
        print(f"Created thread, thread ID: {thread.id}")

        message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
        print(f"Created message, message ID: {message.id}")

        run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=agent.id)

        # Poll the run while run status is queued or in progress
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)  # Wait for a second
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            print(f"Run status: {run.status}")

        client.beta.assistants.delete(agent.id)
        print("Deleted agent")

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(f"Messages: {messages}")

Created agent, agent ID: asst_R7GZTubuLZ9EEm1bwlYVXQY0
Created thread, thread ID: thread_PWrHOShSECPzcUwl715vnWjl
Created message, message ID: msg_u7xFqjf8d0K82BhjHJGOQt3b
Run status: completed
Deleted agent
Messages: SyncCursorPage[Message](data=[Message(id='msg_fwDnY7rd4iAdRyFrYE6bScV4', assistant_id='asst_R7GZTubuLZ9EEm1bwlYVXQY0', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Sure! How about this one:\n\nWhy don't skeletons fight each other?\n\nThey don't have the guts."), type='text')], created_at=1735236468, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_runswyo6lAiB10GcMMjx0P1h', status=None, thread_id='thread_PWrHOShSECPzcUwl715vnWjl'), Message(id='msg_u7xFqjf8d0K82BhjHJGOQt3b', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Hello, tell me a joke'), type='text')], created_at=1735236466, incomplete_at=